<a href="https://colab.research.google.com/github/OheneBoateng/HTM737-OHENE/blob/main/Week11%20code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing the dataset**

**Here's a text block where I don't need to type # before commenting**

In [1]:
import pandas as pd

df = pd.read_csv('https://www.dropbox.com/scl/fi/of5glv327d7wrgstpe8fc/df_ed.csv?rlkey=0m58ktluzuf6d439zjnv035da&dl=1')

print(f"Dataset: {df.shape[0]} rows/observations, {df.shape[1]} columns/fields")
df.head()# loading and reviewing top 5 from the cdc data

/tmp/ipython-input-3288901477.py:3: DtypeWarning: Columns (52,191,513,533,553) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('https://www.dropbox.com/scl/fi/of5glv327d7wrgstpe8fc/df_ed.csv?rlkey=0m58ktluzuf6d439zjnv035da&dl=1')


Dataset: 24777 rows/observations, 579 columns/fields


,VMONTH,VDAYR,ARRTIME,WAITTIME,LOV,AGE,AGER,AGEDAYS,RESIDNCE,SEX,...,RX12V3C1,RX12V3C2,RX12V3C3,RX12V3C4,SETTYPE,YEAR,CSTRATM,CPSUM,PATWT,EDWT
0,1,3,647,33,58,46,4,-7,1,2,...,NaN,NaN,NaN,NaN,3,2013,20113201,100020,2945,NaN
1,1,3,1841,109,150,56,4,-7,1,2,...,NaN,NaN,NaN,NaN,3,2013,20113201,100020,2945,NaN
2,1,3,1333,84,198,37,3,-7,1,2,...,NaN,NaN,NaN,NaN,3,2013,20113201,100020,2945,NaN
3,1,3,1401,159,276,7,1,-7,1,1,...,NaN,NaN,NaN,NaN,3,2013,20113201,100020,2945,NaN
4,1,4,1947,114,248,53,4,-7,1,1,...,NaN,NaN,NaN,NaN,3,2013,20113201,100020,2945,NaN


# **Creating the target variable**

In [3]:

response_cols = ['ADMITHOS','TRANOTH','TRANPSYC','OBSHOS','OBSDIS']

df['ADMITFINAL'] = (df[response_cols].sum(axis=1) >= 1).astype(int)


df = df.drop(response_cols, axis=1)

print("Admission rate:")
print(df['ADMITFINAL'].value_counts())
print(f"\n{df['ADMITFINAL'].mean():.1%} were admitted")#combines and focuses admission we target

Admission rate:
ADMITFINAL
0    21358
1     3419
Name: count, dtype: int64

13.8% were admitted


In [4]:
df['ADMITFINAL'].head()

,ADMITFINAL
0,0
1,0
2,0
3,0
4,1


# **Invalid values removal**

In [7]:
print("WAITTIME distribution:")
print(df['WAITTIME'].value_counts().head(20))#prints 20 of the targeted variable


WAITTIME distribution:
WAITTIME
0     673
10    513
8     511
6     509
5     505
7     495
4     465
9     458
12    442
15    439
11    435
13    428
17    416
16    414
14    411
1     404
3     401
2     386
18    382
21    352
Name: count, dtype: int64


In [6]:

df['WAITTIME'] = df['WAITTIME'].replace([-7, -9], None)

print("After fixing:")
print(df['WAITTIME'].describe())
print(f"\nMissing: {df['WAITTIME'].isnull().sum()}")#we fix the invalid negative integer by replacing with Nan, however this creates an empty value problem we fix below

After fixing:
count     20469
unique      479
top           0
freq        673
Name: WAITTIME, dtype: int64

Missing: 4308


# **Missing values imputation**

**A- Mean imputation** (for continuous variable)

In [8]:

mean_wait = df['WAITTIME'].mean()
print(f"Average wait time: {mean_wait:.1f} minutes")#we fix empty values by filling with average wait time

df['WAITTIME'] = df['WAITTIME'].fillna(mean_wait)

print(f"Missing after: {df['WAITTIME'].isnull().sum()}")



Average wait time: 45.9 minutes
Missing after: 0


/tmp/ipython-input-3875831790.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['WAITTIME'] = df['WAITTIME'].fillna(mean_wait)


In [9]:
df['WAITTIME'] = df['WAITTIME'].fillna(mean_wait).infer_objects(copy=False)

**B- Mode imputation** (for categorical variables)
Mode = the most repeated/common value

In [10]:

print("SEX distribution:")
print(df['SEX'].value_counts())

mode_sex = df['SEX'].mode()[0]
df['SEX'] = df['SEX'].fillna(mode_sex)


SEX distribution:
SEX
1    13702
2    11075
Name: count, dtype: int64


# **Feature Engineering**
Create a new feature using existing data


In [11]:
df['WINTER'] = df['VMONTH'].isin([12, 1, 2, 3]).astype(int)#creates new feature like winter by listing months which define winter

print("Winter visits:")
print(df['WINTER'].value_counts())#from previously defined winter feature, prints count

Winter visits:
WINTER
0    16584
1     8193
Name: count, dtype: int64


In [13]:
df['NIGHT'] = ((df['ARRTIME'] < 800) | (df['ARRTIME'] >= 2000)).astype(int)#create night feature based on arrival time

print(f"{df['NIGHT'].mean():.1%} arrive at night")

33.8% arrive at night


# **Drop unnecessary columns**

In [14]:

df = df.drop(['VMONTH', 'ARRTIME', 'VDAYR'], axis=1)


df = df.drop(['PATCODE', 'HOSPCODE'], axis=1)


df = df.drop(['AGEDAYS'], axis=1)

print(f"Columns remaining: {df.shape[1]}")

#simply drops unneeded colums like one features wer created

Columns remaining: 571


In [15]:
df['ADMITFINAL'].head()

,ADMITFINAL
0,0
1,0
2,0
3,0
4,1


# **One Hot Encoding**

In [18]:

search_word = 'diag'
for column in df.columns:
    if search_word in column.lower():
        print(column)#identify columns relevant for one hot encoding

DIAG2
DIAG3
PRDIAG1
PRDIAG2
PRDIAG3
DIAG1R
DIAG2R
DIAG3R
DIAGSCRN
TOTDIAG
HDDIAG1
HDDIAG2
HDDIAG3
HDDIAG1R
HDDIAG2R
HDDIAG3R
DIAG1_0059-
DIAG1_0079-
DIAG1_00845
DIAG1_0085-
DIAG1_00863
DIAG1_0088-
DIAG1_0090-
DIAG1_0091-
DIAG1_0092-
DIAG1_0093-
DIAG1_01190
DIAG1_0340-
DIAG1_0341-
DIAG1_0380-
DIAG1_0382-
DIAG1_0383-
DIAG1_03842
DIAG1_0389-
DIAG1_04100
DIAG1_04110
DIAG1_04112
DIAG1_0412-
DIAG1_04184
DIAG1_04186
DIAG1_0419-
DIAG1_042--
DIAG1_0479-
DIAG1_0529-
DIAG1_0538-
DIAG1_0539-
DIAG1_05410
DIAG1_0546-
DIAG1_0549-
DIAG1_0570-
DIAG1_0578-
DIAG1_0579-
DIAG1_0669-
DIAG1_0701-
DIAG1_07070
DIAG1_0740-
DIAG1_0743-
DIAG1_0748-
DIAG1_075--
DIAG1_07799
DIAG1_0780-
DIAG1_07811
DIAG1_07889
DIAG1_0792-
DIAG1_0796-
DIAG1_07989
DIAG1_07998
DIAG1_07999
DIAG1_08881
DIAG1_0980-
DIAG1_09889
DIAG1_09940
DIAG1_0998-
DIAG1_0999-
DIAG1_1100-
DIAG1_1103-
DIAG1_1104-
DIAG1_1105-
DIAG1_1109-
DIAG1_1119-
DIAG1_1120-
DIAG1_1121-
DIAG1_1122-
DIAG1_1123-
DIAG1_11284
DIAG1_11289
DIAG1_1129-
DIAG1_1220-
DIAG1_1274-

In [16]:
print("Before encoding:")
print(df[['DIAG1']].value_counts())

Before encoding:
DIAG1
78900    880
78650    640
4659-    510
5990-    418
7840-    363
        ... 
V7263      1
V7283      1
V7284      1
V8289      1
0570-      1
Name: count, Length: 1935, dtype: int64


In [17]:
df = pd.get_dummies(df, columns=['DIAG1'], prefix='DIAG1', drop_first=True)

In [19]:

search_word = 'diag1'
matching_columns = [column for column in df.columns if search_word in column.lower()]
print(matching_columns)#print columns that match diagnosis 1

['PRDIAG1', 'DIAG1R', 'HDDIAG1', 'HDDIAG1R', 'DIAG1_0059-', 'DIAG1_0079-', 'DIAG1_00845', 'DIAG1_0085-', 'DIAG1_00863', 'DIAG1_0088-', 'DIAG1_0090-', 'DIAG1_0091-', 'DIAG1_0092-', 'DIAG1_0093-', 'DIAG1_01190', 'DIAG1_0340-', 'DIAG1_0341-', 'DIAG1_0380-', 'DIAG1_0382-', 'DIAG1_0383-', 'DIAG1_03842', 'DIAG1_0389-', 'DIAG1_04100', 'DIAG1_04110', 'DIAG1_04112', 'DIAG1_0412-', 'DIAG1_04184', 'DIAG1_04186', 'DIAG1_0419-', 'DIAG1_042--', 'DIAG1_0479-', 'DIAG1_0529-', 'DIAG1_0538-', 'DIAG1_0539-', 'DIAG1_05410', 'DIAG1_0546-', 'DIAG1_0549-', 'DIAG1_0570-', 'DIAG1_0578-', 'DIAG1_0579-', 'DIAG1_0669-', 'DIAG1_0701-', 'DIAG1_07070', 'DIAG1_0740-', 'DIAG1_0743-', 'DIAG1_0748-', 'DIAG1_075--', 'DIAG1_07799', 'DIAG1_0780-', 'DIAG1_07811', 'DIAG1_07889', 'DIAG1_0792-', 'DIAG1_0796-', 'DIAG1_07989', 'DIAG1_07998', 'DIAG1_07999', 'DIAG1_08881', 'DIAG1_0980-', 'DIAG1_09889', 'DIAG1_09940', 'DIAG1_0998-', 'DIAG1_0999-', 'DIAG1_1100-', 'DIAG1_1103-', 'DIAG1_1104-', 'DIAG1_1105-', 'DIAG1_1109-', 'DIAG1_111

In [20]:
df['DIAG1_78900'].describe()

,DIAG1_78900
count,24777
unique,2
top,False
freq,23897


# **Splitting the data into train and test**

In [21]:
from sklearn.model_selection import train_test_split


X = df.drop(['ADMITFINAL'], axis=1)
y = df['ADMITFINAL']


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

print(f"Training: {len(X_train):,} patients")
print(f"Testing: {len(X_test):,} patients")#splits into training=75% and testing=25%

Training: 18,582 patients
Testing: 6,195 patients


In [22]:
X.head()

,WAITTIME,LOV,AGE,AGER,RESIDNCE,SEX,ETHUN,ETHIM,RACEUN,RACER,...,DIAG1_V829-,DIAG1_V8712,DIAG1_V8719,DIAG1_V872-,DIAG1_V8739,DIAG1_V990-,DIAG1_V991-,DIAG1_V992-,DIAG1_V997-,DIAG1_V998-
0,33.0,58,46,4,1,2,1,1,1,1,...,False,False,False,False,False,False,False,False,False,False
1,109.0,150,56,4,1,2,1,1,1,1,...,False,False,False,False,False,False,False,False,False,False
2,84.0,198,37,3,1,2,1,1,1,1,...,False,False,False,False,False,False,False,False,False,False
3,159.0,276,7,1,1,1,1,1,1,1,...,False,False,False,False,False,False,False,False,False,False
4,114.0,248,53,4,1,1,1,1,1,1,...,False,False,False,False,False,False,False,False,False,False


In [23]:
y.head()

,ADMITFINAL
0,0
1,0
2,0
3,0
4,1


In [24]:
X_train.shape

(18582, 2503)

In [25]:
X_test.shape

(6195, 2503)